In [5]:
import requests
url= 'https://povapi-ig6qgbx3oa-ew.a.run.app/predict?'
params = dict(lat=46.2332,lon=6.1232)
response = requests.get(url, params=params)
prediction= response.json()
pi = prediction['value']

JSONDecodeError: [Errno Expecting value] Service Unavailable: 0

In [3]:
import streamlit as st
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import folium
from streamlit_folium import folium_static
from folium.plugins import Fullscreen
from PIL import Image
import ee
import os
from google.oauth2 import service_account
from dotenv import load_dotenv, find_dotenv
import json
from haversine import inverse_haversine, Direction
import time
import requests

In [4]:
service_account = 'annuka@poverty-mapper.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '/Users/annuka/Downloads/poverty-mapper-deef500638d8.json')
ee.Initialize(credentials)


In [6]:


#function with API, Population count based on gridded population
def frontend_manipulation(lat,lon, radius_):    
    
    # #API URL and poverty index input
    # url= 'https://povapi-ig6qgbx3oa-ew.a.run.app/predict?'
    # params = dict(lat=lat,lon=lon)
    # response=requests.get(url, params=params)
    # prediction= response.json()
    # pi = prediction['value']
    
    #pi placeholder
    pi=40
    #Population count based on lat, lon
    pop_image_collection= ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count")
    #scale remains constant
    scale = 1000
    #in the case there is no radius value entered by user, it will automatically be 1
    if radius_ == '':
        radius_=1
    #grid_size in km
    grid_size =int(f'{radius_}')
    radius=(grid_size * 2**0.5 )/2

    coord_ne = inverse_haversine((lat,lon), radius, Direction.NORTHEAST)
    coord_sw = inverse_haversine((lat,lon), radius, Direction.SOUTHWEST)

    coordinates_rectangle = ee.Geometry.Rectangle([
                                [coord_sw[1], coord_sw[0]],
                                [coord_ne[1], coord_ne[0]]
                                ])

    pop_region = pop_image_collection.select('population_count').getRegion(coordinates_rectangle, scale).getInfo()

    df = pd.DataFrame(pop_region)
    df.columns = df.iloc[0]
    df = df[1:]
    df = df.sort_values('time')
    df.reset_index(inplace=True, drop=True)
    #'1577836800000' is the value for 2020 data
    df = df.loc[df['time'] == 1577836800000]

    #variables to input in interactive map  
    population_count= int(df['population_count'].sum())
    pop_pov_line = int(population_count*(pi/100))
    
    #st.text(f'Location: {city}, {province}, {country}. lat, lon: {lat}, {lon}')
    st.text(f'Latitude: {lat}     Longitude: {lon}    Radius: {radius_} km')
    st.text(f'Poverty Index : {pi}%')
    st.text(f'Total population(2020): {population_count}')
    st.text(f'Population living below the poverty line: {pop_pov_line}') 

In [42]:
lats= []
lons = []
radii = []

In [43]:
lat = input('please enter latitude')
lats.append(lat)
lon = input ('please enter longitude')
lons.append(lon)
radius_= input ('please enter radius in km')
radii.append(radius_)

please enter latitude 115.1385
please enter longitude 8.6478
please enter radius in km 5000


In [12]:
token = 'pk.eyJ1Ijoia3Jva3JvYiIsImEiOiJja2YzcmcyNDkwNXVpMnRtZGwxb2MzNWtvIn0.69leM_6Roh26Ju7Lqb2pwQ'  # your mapbox token
tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(token)


In [35]:
map_final = folium.Map(location=[lat,lon],zoom_start=10,tiles=tileurl, attr='Mapbox', control_scale=False) 
Fullscreen().add_to(map_final)

In [46]:
map_final=folium.Map(location=[8.6478,115.1385], zoom_start=11.25)
for lat,lon,radius in zip(lats, lons, radii):
        # sets some variables from the df
        
# plots the CircleMarker
    folium.CircleMarker([lat, lon], radius=radius, color='#3186cc', fill_color='#3186cc',fill=True, fill_opacity= .7).add_to(map_final)
    # saves a map with all the circle markers in a row
   # map_final=5

map_final.save("/Users/annuka/code/karimstephan/povmapper/annuka/"+str(myint)+'.html')
    

In [47]:
map_final